1. verinin hazırlanması 

In [1]:
pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 2.1 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Obtaining dependency information for autograd>=1.2.0 from https://files.pythonhosted.org/packages/81/70/d5c7c2a458b8be96495c8b1634c2155beab58cbe864b7a9a5c06c2e52520/autograd-1.6.2-py3-none-any.whl.metadata
  Obtaining dependency information for dill>=0.2.6 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Using cached future-0.18.3-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/ce/25/a557ee10ac9dce1300850024707ce1850a6958f1673a9194be878b99d631/matplotlib-3.8.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/5b/ac/26fa1057f62beaa2af4c55c6ac733b114a403b746cfe0ce3dc6e4aec921a/contourpy-1.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/14/f9/14494b9b440a8ab0b8746d0c65b25c2a5ba22ed9a946d0895e2e4f39d06f/fonttools-4.47.0-cp311-cp311-macosx_10_9_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 997.0 kB/s eta 0:00:000:00:01m eta 0:00:01
  Obtaining 

In [5]:
import pandas as pd 
import datetime as dt 
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_frequency_recency_matrix

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [7]:
def outlier_tresholds(dataframe, variable):  # aykırı değerleri yakalamak için kullanılan fonksiyon
    quartile1 = dataframe[variable].quantile(0.01) # %1'lik kısım
    quartile3 = dataframe[variable].quantile(0.99) # %99'lik kısım
    interquantile_range = quartile3 - quartile1 # aradaki fark
    up_limit = quartile3 + 1.5 * interquantile_range # üst sınır
    low_limit = quartile1 - 1.5 * interquantile_range # alt sınır
    return low_limit, up_limit # sınırları döndürür

In [8]:
def replace_with_thresholds(dataframe, variable): # aykırı değerleri sınırlarla değiştirmek için kullanılan fonksiyon
    low_limit, up_limit = outlier_tresholds(dataframe, variable) # sınırları alır
    #dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit # aykırı değerleri sınırlara göre değiştirir.
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit # aykırı değerleri sınırlara göre değiştirir.


In [12]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df  = df_.copy()
df.head()



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


veri ön işleme

In [13]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [14]:
df.dropna(inplace=True)

In [15]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,406830.0000,12.0613,-80995.0000,2.0000,5.0000,12.0000,80995.0000,248.6931
InvoiceDate,406830,2011-07-10 16:31:30.127424512,2010-12-01 08:26:00,2011-04-06 15:02:00,2011-07-31 11:48:00,2011-10-20 13:06:00,2011-12-09 12:50:00,NaN
Price,406830.0000,3.4605,0.0000,1.2500,1.9500,3.7500,38970.0000,69.3151
Customer ID,406830.0000,15287.6842,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000,1713.6031


In [17]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [20]:
df = df[~df["Invoice"].str.contains("C", na=False)] # iade olanları çıkardık


AttributeError: Can only use .str accessor with string values!

In [21]:
df = df[df["Quantity"] > 0]  # 0 adet olanları çıkardık
df = df[df["Price"] > 0]  # 0 fiyatlı olanları çıkardık

In [24]:
replace_with_thresholds(df, "Quantity") # aykırı değerleri sınırlarla değiştirdik

In [25]:
replace_with_thresholds(df, "Price") # aykırı değerleri sınırlarla değiştirdik

In [26]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0000,11.8308,1.0000,2.0000,6.0000,12.0000,298.5000,25.5231
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0000,2.8935,0.0010,1.2500,1.9500,3.7500,37.0600,3.2272
Customer ID,397885.0000,15294.4169,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000,1713.1444


In [39]:
# drop invoice date 
df.drop("InvoiceDate", axis=1, inplace=True)

KeyError: "['InvoiceDate'] not found in axis"

In [40]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,11.8308,25.5231,1.0000,2.0000,6.0000,12.0000,298.5000
Price,397885.0000,2.8935,3.2272,0.0010,1.2500,1.9500,3.7500,37.0600
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000
TotalPrice,397885.0000,20.6298,51.8285,0.0010,4.6800,11.8000,19.8000,3268.5750


In [42]:
df["TotalPrice"] = df["Quantity"] * df["Price"] # toplam fiyatı hesapladık

In [43]:
today_date = dt.datetime(2011, 12, 11) # bugünün tarihini belirledik

#lifetime veri yapısının hazırlanması 

In [44]:
# recency, frequency, monetary
# recency: müşterinin son satın alma tarihinden bugüne kadar geçen süre yerine müşterinin ilk satın alma tarihinden bugüne kadar geçen süre kullanılır.
# T:  Müşterinin ilk satın alma tarihinden bugüne kadar geçen süre.
# frequency: tekrar eden toplam satın alma sayısı.
# monetary: satın alama başına ortalama kazanç.

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                                         lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
                                         'Invoice': lambda Invoice: Invoice.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()
                                         })
cltv_df.columns = cltv_df.columns.droplevel(0)

cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
# satın alama başına ortalama kazanç.
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
# frequency 1'den büyük olanları alır.
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df["recency"] = cltv_df["recency"] / 7  # recency'yi haftaya çevirdik
cltv_df["T"] = cltv_df["T"] / 7  # T'yi haftaya çevirdik


KeyError: "Column(s) ['InvoiceDate'] do not exist"

In [46]:
bgf = BetaGeoFitter(penalizer_coef=0.001) # modeli kurduk

bgf.fit(cltv_df['frequency'],
       cltv_df['recency'],
       cltv_df['T'])



NameError: name 'cltv_df' is not defined

In [49]:
# 4 hagta içinde en çok satın alma beklediğimiz 10 müşteri
bgf.conditional_expected_number_of_purchases_up_to_time(4, # 1 hafta
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']).sort_values(ascending=False).head(10)


NameError: name 'cltv_df' is not defined

In [47]:
bgf.predict(1, # 1 hafta
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)



AttributeError: 'BetaGeoFitter' object has no attribute 'predict'

In [50]:
# 3 gamma - gamma modeli 

ggf = GammaGammaFitter(penalizer_coef=0.01) # modeli kurduk
ggf.fit(clctv_df['frequency'], cltv_df['monetary']) # modeli fit ettik
ggf.conditional_expected_average_profit(clctv_df['frequency'], cltv_df['monetary']).sort_values(ascending=False).head(10) # ortalama karı hesapladık
ggf.plot_conditional_expected_average_profit(clctv_df['frequency'], cltv_df['monetary']) # ortalama karı grafiği

NameError: name 'clctv_df' is not defined